# Week 3 Homework — Processes, Threads, and Concurrency

Instructions:
- Implement and analyze a thread pool processing pipeline.
- Provide code, outputs, and concise analysis for each task.
- Use Python 3.9+.

Grading focuses on correctness, safe concurrency, sensible measurement, and clarity of interpretation.

---

## Task 1 — Build a configurable thread pool manager
Implement a `ThreadPool` with:
- Fixed number of worker threads
- Bounded queue for tasks (configurable capacity)
- `submit(fn, *args, **kwargs)` to enqueue work
- `shutdown(wait=True)` for graceful termination

Demonstrate submitting 1000 no-op tasks and waiting for completion.


In [ ]:
import threading, queue, time
from typing import Callable


class ThreadPool:
    def __init__(self, workers: int = 4, max_queue: int = 100):
        self.tasks = queue.Queue(maxsize=max_queue)
        self.workers = []
        self.running = True
        for _ in range(workers):
            t = threading.Thread(target=self._worker, daemon=True)
            t.start()
            self.workers.append(t)

    def _worker(self):
        while self.running:
            try:
                fn, args, kwargs = self.tasks.get(timeout=0.2)
            except queue.Empty:
                continue
            try:
                fn(*args, **kwargs)
            finally:
                self.tasks.task_done()

    def submit(self, fn: Callable, *args, **kwargs):
        self.tasks.put((fn, args, kwargs))

    def shutdown(self, wait=True):
        self.running = False
        if wait:
            for t in self.workers:
                t.join(timeout=1)

# Demo
pool = ThreadPool(workers=4, max_queue=50)
for _ in range(1000):
    pool.submit(lambda: None)
pool.tasks.join()
pool.shutdown()
print('Task 1: OK')


## Task 2 — Measure I/O-bound throughput and latency vs thread count
Simulate an I/O-bound workload using `time.sleep()` to represent external waits.

Procedure:
- For thread counts: 1, 2, 4, 8, 16
- Enqueue 2000 tasks, each sleeping an exponentially distributed delay with mean 5 ms
- Measure: total time, throughput (tasks/sec), and average per-task service time

Report the results in a small table or printed dicts and briefly interpret scaling behavior.


In [ ]:
import random

class Metrics:
    def __init__(self):
        self.count = 0
        self.total = 0.0
        self.lock = threading.Lock()

    def record(self, dt):
        with self.lock:
            self.count += 1
            self.total += dt


def io_task(metrics: Metrics, mean_ms=5.0):
    t0 = time.perf_counter()
    time.sleep(random.expovariate(1.0/mean_ms) / 1000.0)
    metrics.record(time.perf_counter() - t0)


def bench_io(workers_list=(1,2,4,8,16), tasks=2000, mean_ms=5.0):
    results = []
    for w in workers_list:
        pool = ThreadPool(workers=w)
        m = Metrics()
        t0 = time.perf_counter()
        for _ in range(tasks):
            pool.submit(io_task, m, mean_ms)
        pool.tasks.join()
        elapsed = time.perf_counter() - t0
        pool.shutdown()
        avg_ms = (m.total/m.count)*1e3 if m.count else 0
        results.append({'workers': w, 'elapsed_s': elapsed, 'throughput_rps': tasks/elapsed, 'avg_ms': avg_ms})
    return results

for r in bench_io():
    print(r)


### Analysis (Task 2)
Write 3–6 sentences:
- How does throughput change with thread count? Where do returns diminish and why?
- Why doesn’t average per-task service time change much across thread counts?

---

## Task 3 — Queueing and backpressure
Set the queue capacity to a small value (e.g., 10). Measure enqueue wait time when producing faster than workers can consume.

Implementation hints:
- Measure time spent inside `submit` (waiting for a free slot)
- Run with workers=2, tasks=1000, mean sleep ≈ 5 ms
- Report total enqueue wait time and p50/p95 enqueue wait (if you collect samples)


In [ ]:
import statistics

def bench_backpressure(workers=2, max_queue=10, tasks=1000, mean_ms=5.0):
    pool = ThreadPool(workers=workers, max_queue=max_queue)
    enqueue_waits = []

    def task():
        time.sleep(random.expovariate(1.0/mean_ms) / 1000.0)

    t0 = time.perf_counter()
    for _ in range(tasks):
        s0 = time.perf_counter()
        pool.submit(task)
        enqueue_waits.append(time.perf_counter() - s0)
    pool.tasks.join()
    elapsed = time.perf_counter() - t0
    pool.shutdown()

    waits_ms = [w*1e3 for w in enqueue_waits]
    waits_ms.sort()
    p50 = waits_ms[int(0.5*(len(waits_ms)-1))]
    p95 = waits_ms[int(0.95*(len(waits_ms)-1))]
    return {'elapsed_s': elapsed, 'total_enqueue_wait_s': sum(enqueue_waits), 'p50_enqueue_ms': p50, 'p95_enqueue_ms': p95}

print(bench_backpressure())


### Analysis (Task 3)
Explain how bounded queues implement backpressure and why enqueue wait grows when producers outrun consumers.

---

## Task 4 — CPU-bound caution (thought + optional code)
Explain why Python threads provide limited speedup for CPU-bound loops (the GIL). Optionally, write a short experiment comparing threads vs `multiprocessing` for a CPU task.

Write 4–8 sentences and include code if you run the comparison.
